In [4]:
import numpy as np

In [8]:

semantic_dim = 360*90


In [2]:
point_grid_path = "./out/build/grid_points.dat"
index_map_path = "./out/build/index_map.dat"
shadow_map_path = "./out/build/shadow_map.dat"

azimuth_map_path = "./out/build/azimuth_map.dat"
elevation_map_path = "./out/build/elevation_map.dat"

In [5]:
index_map = np.memmap(index_map_path, dtype=np.uint32, mode='r')
azimuth_map = np.memmap(azimuth_map_path, dtype=np.float16, mode='r')
elevation_map = np.memmap(elevation_map_path, dtype=np.float16, mode='r')

(105,)


In [9]:




sparse_voxel = {}

point_grid_num = 331547
bbox_min = (84501.6, 445805, -3.747)
resolution = 2.0
voxel_dim = (587,590,50)

point_grid = np.array((point_grid_num, 6), dtype=np.float32)
point_attribute = np.array((point_grid_num, 1), dtype=np.int32)
point_grid_irradiance = np.array((point_grid_num, 1), dtype=np.float32)


for i in range(point_grid_num):
    coords = point_grid[i, 0:3]
    normal = point_grid[i, 3:6]
    ix = int((coords[0] - bbox_min[0]) / resolution)
    iy = int((coords[1] - bbox_min[1]) / resolution)
    iz = int((coords[2] - bbox_min[2]) / resolution)
    voxel_idx = ix + iy * voxel_dim[0] + iz * voxel_dim[0] * voxel_dim[1]
    if voxel_idx not in sparse_voxel:
        sparse_voxel[voxel_idx] = []
    sparse_voxel[voxel_idx].append(normal)
    sparse_voxel[voxel_idx].append(point_grid_irradiance[i])



for voxel in sparse_voxel:
    data = np.array(sparse_voxel[voxel])

    num_points = data.shape[0]/2
    normals = data[::2]
    irradiance_values = data[1::2]

    elevations = np.degrees(np.arcsin(normals[:, 2]))
    azimuths = np.degrees(np.arctan2(normals[:, 1], normals[:, 0]))

    intensity_up_ratio = np.sum(elevations > 45) / num_points
    intensity_down_ratio = np.sum(elevations < -45) / num_points
    intensity_front_ratio = np.sum((elevations > 0) & (elevations <= 45) & (azimuths >= -135) & (azimuths <= -45)) / num_points
    intensity_back_ratio = np.sum((elevations > 0) & (elevations <= 45) & (azimuths >= 45) & (azimuths <= 135)) / num_points
    intensity_left_ratio = np.sum((elevations > 0) & (elevations <= 45) & ((azimuths >= 135) | (azimuths <= -135))) / num_points
    intensity_right_ratio = np.sum((elevations > 0) & (elevations <= 45) & (azimuths >= -45) & (azimuths <= 45)) / num_points

    sparse_voxel[voxel].append([intensity_up_ratio, intensity_down_ratio, intensity_front_ratio, intensity_back_ratio, intensity_left_ratio, intensity_right_ratio])

In [ ]:
def iterative_calculation(point_grid_irradiance, sparse_voxel):
    